# Notebook rilut

### Bikin jelek, hapus aja

- colorCode, total_spent, customerID, articleID, orderDate, productGroup

### Jangan dihapus

- price, rrp (walaupun impute median)

### Missing values

- rrp, productGroup

### Catatan:

Logistic Regression lama banget. itu pun solvernya udah diubah. kalau masih mau pake, iternya kurangin kali?

### TODO:

- ekstrak fitur karena fitur adalah segalanya

In [23]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [24]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue2.csv')

if 'deviceID' in df.columns:
    del df['deviceID']
    
del df["colorCode"]
del df['customerID']
del df['articleID']
del df['orderDate']
#del df['productGroup']

    
if 'orderID' in df.columns:
    del df['orderID']

df.head()

,paymentMethod,price,productGroup,quantity,returnQuantity,rrp,sizeCode,voucherAmount,order_total,customer_budget,...,after_voucher,order_order,choice_order,articleID_cumprob,colorCode_cumprob,customerID_cumprob,sizeCode_cumprob,ac_cumprob,as_cumprob,cps_cumprob
0,BPRG,10.000000,3.0,1,0.0,29.990000,44,0.0,30.000000,28.500000,...,30.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BPRG,20.000000,3.0,1,0.0,39.990002,44,0.0,30.000000,28.500000,...,30.000000,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BPRG,35.000000,8.0,1,0.0,49.990002,38,0.0,84.989998,84.989998,...,84.989998,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BPRG,49.990002,8.0,1,1.0,49.990002,38,0.0,84.989998,84.989998,...,84.989998,2,2,1.0,1.0,0.5,0.5,1.0,1.0,0.5
4,PAYPALVC,10.000000,8.0,1,0.0,35.990002,42,0.0,60.000000,60.000000,...,60.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# Turunin presisi, iseng-iseng aja
float_64_columns = df.loc[:, df.dtypes == np.float64].columns
for col in float_64_columns:
    df[col] = df[col].astype(np.float32)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    
object_columns=df.loc[:, df.dtypes == object].columns
df= df.drop(object_columns, axis=1)


label='returnQuantity'
df.rrp.fillna(df.rrp.median(), inplace=True)
Xdf=df.drop(['productGroup','voucherAmount','after_voucher'], axis=1)
Xdf=Xdf.drop([label], axis=1)

X=Xdf.values
y=df.returnQuantity.values


X_train=X[:-341099]
X_test=X[-341099:]
y_train=y[:-341099]
y_test=y[-341099:]

In [49]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)
Xdf.columns.tolist()

[[142896  24015      8      0      0      0]
 [ 19149 154353      8      0      0      0]
 [     3    618      9      0      0      0]
 [     0     11     18      0      0      0]
 [     0      0      7      0      0      0]
 [     0      0      4      0      0      0]]
43878.0
jumlah data test:  341099


['price',
 'quantity',
 'rrp',
 'order_total',
 'customer_budget',
 'expense_ratio',
 'unit_price',
 'median_unit_price',
 'price_diff',
 'order_order',
 'choice_order',
 'articleID_cumprob',
 'colorCode_cumprob',
 'customerID_cumprob',
 'sizeCode_cumprob',
 'ac_cumprob',
 'as_cumprob',
 'cps_cumprob']

# 43878
Atas->Bawah: True, Kiri->Kanan: Predicted
```
[[142896  24015      8      0      0      0]
 [ 19149 154353      8      0      0      0]
 [     3    618      9      0      0      0]
 [     0     11     18      0      0      0]
 [     0      0      7      0      0      0]
 [     0      0      4      0      0      0]]
43878.0
jumlah data test:  341099
Out[49]:
['price',
 'quantity',
 'rrp',
 'order_total',
 'customer_budget',
 'expense_ratio',
 'unit_price',
 'median_unit_price',
 'price_diff',
 'order_order',
 'choice_order',
 'articleID_cumprob',
 'colorCode_cumprob',
 'customerID_cumprob',
 'sizeCode_cumprob',
 'ac_cumprob',
 'as_cumprob',
 'cps_cumprob']
```

### Ini Importances?

In [29]:
"""from sklearn.feature_selection import RFE
lr=LinearRegression(n_jobs=-1, normalize=True)
selector = RFE(lr, 15, step=1)
selector = selector.fit(X, y)
pair=zip(Xdf.columns, selector.ranking_)
t1=[i for i in pair]; t1
sorted(t1, key=itemgetter(1))"""

'from sklearn.feature_selection import RFE\nlr=LinearRegression(n_jobs=-1, normalize=True)\nselector = RFE(lr, 15, step=1)\nselector = selector.fit(X, y)\npair=zip(Xdf.columns, selector.ranking_)\nt1=[i for i in pair]; t1\nsorted(t1, key=itemgetter(1))'

```

```